**Задание на зачет:**

У вас есть документ: Info.xlsx

Вам нужно на python написать скрипт, с помощью которого генерируется 1000 разных pdf файлов по строкам из Info.xlsx.

Вид итогового документа:

![](example.png)

Способ формирования документа:

Из xlsx файла убирается случайным образом от (0; 30] % строк. 

* Одна строка – это набор значений из колонок A-G из исходного xlsx документа (набор значений не меняется) 
* +3 дополнительных сгенеренных значения для колонок **'Ориентировочный объем', 'Стоимость за единицу', 'Всего'**:

**1. Ориентировочный объем** – это случайное целое значение в интервале от 1 до 100

**2. Стоимость** – случайное числовое значение [0 ; 1000000], округление до 2-го знака (до копейки). В итоговом документе должно быть [10 ; 40]% нулевых значений, 50% значений: (0 ; 100000], остальные – со значениями больше 100000

**3. Всего** – произведение объема и стоимости.

Convert Excel to PDF in Python | XLSX to PDF, XLS to PDF in Python

https://blog.aspose.com/2021/04/02/convert-excel-files-to-pdf-in-python/

In [1]:
#!pip install xlsxwriter
#!pip install aspose-cells
#!pip install PyPDF2
#!pip install pymupdf

In [2]:
import pandas as pd
import numpy as np
import random
import os

import jpype
import asposecells
jpype.startJVM()
from asposecells.api import Workbook, SaveFormat, PdfSaveOptions, PageOrientationType

https://pythonru.com/uroki/osnovy-pandas-3-vazhnye-metody-formatirovanija-dannyh#:~:text=%D0%A1%D0%BF%D0%BE%D1%81%D0%BE%D0%B1%D1%8B%20%D0%BE%D0%B1%D1%8A%D0%B5%D0%B4%D0%B8%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F%3A%20inner%2C%20outer%2C%20left%2C%20right&text=%D0%92%20SQL%20%D0%B5%D1%81%D1%82%D1%8C%204%20%D1%82%D0%B8%D0%BF%D0%B0,%D0%BC%D0%BE%D0%B6%D0%BD%D0%BE%20%D0%BD%D0%B0%D0%B9%D1%82%D0%B8%20%D0%B2%20%D0%BE%D0%B1%D0%B5%D0%B8%D1%85%20%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0%D1%85.

### 1. Загрузка

In [3]:
cols = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'] #, 'K', 'L']
head = pd.read_excel('Info.xlsx', names=cols)[:6]
head['I'][4]  = 'Стоимость за единицу, руб'
head['J'][4]  = 'Итого стоимость, руб'

In [4]:
# Load the xlsx file
data = pd.read_excel('Info.xlsx', header=6,  usecols = list(range(9)), names=cols) # index_col = list(range(5)),
#data.head(3)

In [5]:
n,m = np.shape(data)
n,m
data_end = 0
for i in range(n-1,1,-1):
    if data['I'][i] == 'ИТОГО':
        data_end = i
footer = data[:][data_end:]
data = data[:][:data_end]

### 2. Предобработка df

In [6]:
def GetDct(num, name, status, damaged, jobs, unit, serviceNum, volume, cost, total):
    dct = {
        'A':num, 
        'B':name,                  
        'C':status,      
        'D':damaged,     
        'E':jobs,          
        'F':unit,         
        'G':serviceNum,
        'H':volume,       
        'I':cost,           
        'J':total}     
    return dct

In [7]:
num = ''
name = ''
status = ''
damaged = ''
jobs = ''
unit = ''
serviceNum = ''
#volume = 0
#cost = 0
#total = 0

In [8]:
arr=[]
for i in range(0,data_end):
    if not(pd.isna(data['A'][i])):
        num = data['A'][i]
        name = data['B'][i]
        status = data['C'][i]
    if not(pd.isna(data['D'][i])):  
        damaged = data['D'][i]
    jobs = data['E'][i]
    unit = data['F'][i]
    serviceNum = str(int(data['G'][i]))
    dct = GetDct(num, name, status, damaged, jobs, unit, serviceNum, 0,0,0) # volume, cost, total)
    arr.append(dct)
df = pd.DataFrame(arr)
#df

### 3. Обработка

In [9]:
def DOCS(df):
    arr=[]
    p = random.randint(0,30)
    for i in range(0, data_end):
        if random.randint(0, 100) > p: 
            num = df['A'][i]
            name = df['B'][i]
            status = df['C'][i]
            damaged = df['D'][i]
            jobs = df['E'][i]
            unit = df['F'][i]
            serviceNum = str(int(df['G'][i]))
            volume = random.randint(1,100)
            p1 = random.randint(1,10)
            if p1 < 6:                     
                cost = round(random.uniform(0, 1e5), 2)
            elif p1 < random.randint(6,9): 
                cost = 0
            else:                      
                cost = round(random.uniform(1e5, 1e6), 2)
            total = volume * cost 
            dct = GetDct(num, name, status, damaged, jobs, unit, serviceNum, volume, cost, total)
            arr.append(dct)
    newdf = pd.DataFrame(arr)
    return newdf

In [10]:
from pathlib import Path
import fitz
import numpy as np
from PIL import Image

def generate(start, end):
    return random.randint(start, end)

FONST = ['Times New Roman', 'Calibri', 'Aial']
width = [54, 85, 62, 110, 221, 54, 74, 54, 54, 54, 54, 54]

In [13]:
for k in range(100):
    newdf = DOCS(df)
    
    # Склеиваем обратно
    docs = pd.merge(head, newdf, how = 'outer')
    docs = pd.merge(docs, footer, how = 'outer')
    for col in ['A', 'B', 'C', 'D']:
        last_valid = docs.at[6, col]
        for i in range(10, len(docs) - 7):
            if pd.isna(docs.at[i, col]):
                docs.at[i, col] = last_valid
            else:
                last_valid = docs.at[i, col]

    #Формируем Excel
    writer = pd.ExcelWriter(path='output\output{}.xlsx'.format(k))
    docs.to_excel(excel_writer = writer, sheet_name = 'main', header=False, index=False)
    font_name = FONST[generate(0, 2)]
    font_size = generate(9, 12)
    workbook = writer.book
    worksheet = writer.sheets['main']
    worksheet.set_landscape()
    worksheet.fit_to_pages(1, 0) 
    worksheet.set_row(5, 80)
    merge_format = workbook.add_format({'text_wrap': True, 'border': 1, 'align': 'center', 'valign': 'top'})
    for col, col_ind in [('A',0), ('B',1), ('C',2), ('D',3)]:
        startCells = [10]
        for row in range(11, len(docs) - 8):
            if docs.loc[row, col] != docs.loc[row-1, col]:
                startCells.append(row)
        startCells.append(len(docs) - 7)
        for i in range(len(startCells)-1):
            startRow = startCells[i]
            endRow = startCells[i+1]-1;
            if startRow == endRow:
                s = docs.loc[startRow, col] #startRow-1 
                worksheet.write(startRow, col_ind, s , merge_format)
            else:
                s = docs.loc[startRow, col]
                worksheet.merge_range(startRow, col_ind, endRow, col_ind, s , merge_format)
    merge_format.set_font_name(font_name)
    merge_format.set_font_size(font_size)
    for column, w in zip(docs, width):
        col_idx = docs.columns.get_loc(column)
        worksheet.set_column(col_idx, col_idx, w / 6, merge_format)
    cell_format = workbook.add_format({'bg_color': '#deebf6', 'text_wrap': True, 'border': 2, 'bold': True,
                                     'font_name': font_name,'font_size': font_size,'align': 'center','valign': 'vcenter'})
    for i in range(8, 10):
        for j, col in zip(range(len(cols)), cols):
            s = docs.loc[i, col]
            worksheet.write(i, j, s, cell_format)
    for i in range(4,7):
        s = docs.loc[len(docs) - i, 'A']
        worksheet.merge_range(len(docs) - i,  0, len(docs) - i, 3, s)
    merge_format = workbook.add_format({'text_wrap': True, 'align': 'right', 'bold': True,
                                        'valign': 'top','font_size': 15 })
    worksheet.merge_range('A1:J1', 'Приложение '+str(i)+'\nк Техническому заданию', merge_format)
    worksheet.merge_range('A3:J8', 'ПЕРЕЧЕНЬ ПО МЕЛКОМУ РЕМОНТУ', merge_format)
    merge_format = workbook.add_format({'text_wrap': True, 'align': 'right','valign': 'top'})
    worksheet.merge_range('A2:J2', 'к Техническому заданию', merge_format)
    writer.save()    
    
    #Формируем PDF
    workbook = Workbook('output\output{}.xlsx'.format(k))
    workbook.save('output\output{}.pdf'.format(k), SaveFormat.PDF)
    
    #Поворачиваем PDF
    pdffile = 'output/output{}.pdf'.format(str(k))
    doc = fitz.open(pdffile)
    angle = generate(-10, 10) 

    for page_id in range(doc.page_count):
        page = doc.load_page(page_id)
        pix = page.get_pixmap(alpha=False, dpi=154)
        im = np.frombuffer(buffer=pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, 3))
        im = Image.fromarray(im)
        im = im.crop((0, 50, pix.width, pix.height))

        im = im.rotate(angle)
        im.save('_tmp.pdf')

        page_front = fitz.open()
        page_front.insert_pdf(fitz.open('_tmp.pdf'), from_page=page_id, to_page=page_id)
        page.show_pdf_page(page.rect, page_front, pno=0, keep_proportion=True, overlay=True, oc=0, rotate=0, clip=None)

    doc.save('output/pdf_rotated{}.pdf'.format(str(k), encryption=fitz.PDF_ENCRYPT_KEEP))